In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import random
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from pytorch_forecasting.metrics import SMAPE
from captum.attr import IntegratedGradients,NoiseTunnel

/home/sjlee/sj_virtual/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. 모델 구성하기

In [2]:
data = pd.read_csv('./TrafficData/4-Copy1.csv')
data = data['volumn']
sequence_length = 1440
pred_len = 1

In [3]:
from torch import FloatTensor as FloatTensor

In [4]:
def seq_data(x,sequence_length,pred_len):
    seq_list = []
    target_list = []
    device = torch.device("cuda:0")
    for i in range(len(x)-sequence_length-pred_len):
        seq_list.append(x.iloc[i:i+sequence_length].values)
        target_list.append(x.iloc[i+sequence_length:i+sequence_length+pred_len].values)

    return FloatTensor(seq_list).view(-1,1,sequence_length).to(device),FloatTensor(target_list).unsqueeze(1).view(-1,1,pred_len).to(device)

In [5]:
split = 10080
x_train,y_train = seq_data(data.iloc[:-split],sequence_length,pred_len)

/tmp/ipykernel_2417094/4291513198.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return FloatTensor(seq_list).view(-1,1,sequence_length).to(device),FloatTensor(target_list).unsqueeze(1).view(-1,1,pred_len).to(device)


In [6]:
batch_size = 64
train_dataset = TensorDataset(x_train,y_train)
training = DataLoader(train_dataset,batch_size= batch_size)

In [7]:
seed = 42

In [8]:
device = torch.device("cuda:0")

In [9]:
X_test = torch.FloatTensor(data.iloc[-split-sequence_length:-split].values).to(device)
# X_train의 마지막 인덱스에서 sequece_length만큼 뒤에 부분을 잘라오기
X_test = X_test.view(1,1,sequence_length)

In [10]:
target = data.iloc[-split]
target = FloatTensor([target]).view(-1,1)
target

tensor([[90520.]])

## ISSUE
### 1. loss를 Back ward시키기 전에 attribution=0로 수행을 해야하나?
### 2. 모든 index를 추적하면서 관리를 해야하나?
### 3. 앞으로 그 부분을 train 시키지 않기 위한 모델의 function을 만들어야 할듯
&rightarrow; 아래의 reset함수를 수행하면 될 것이다.

## 3번 문제에 대한 진행

In [11]:
A=torch.FloatTensor(np.arange(1440).reshape(-1,1))

In [12]:
B=torch.ones([1440,1])

In [13]:
C=A*B
C[[1,1439]] # Tensor도 Indexing 가능했음

tensor([[1.0000e+00],
        [1.4390e+03]])

In [14]:
C

tensor([[0.0000e+00],
        [1.0000e+00],
        [2.0000e+00],
        ...,
        [1.4370e+03],
        [1.4380e+03],
        [1.4390e+03]])

In [15]:
torch.ones([1,1440])

tensor([[1., 1., 1.,  ..., 1., 1., 1.]])

In [16]:
A = torch.FloatTensor(np.arange(1000)).view(10,1,100)
A[1:4,0,[1,3]] # Tensor Indexing : 만을 사용해서 범위 표시하고
# 만약 A[[1,4],0,[1,3]] 으로 하면 element-wise하게 짝지어진다.

tensor([[101., 103.],
        [201., 203.],
        [301., 303.]])

In [17]:
A = pd.DataFrame({"A":np.arange(1000)})
A.iloc[[1,2,3,4,5]]=np.array([-1,3,-2,-4,-5]).reshape(-1,1) # reshape를 안해주면 size mismatch가 뜬다.
A.values.reshape(-1)

array([  0,  -1,   3,  -2,  -4,  -5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [18]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq_len = 1440
        self.pred_len = 1
        self.first_layer = nn.Linear(self.seq_len,self.pred_len)
        self.first_layer.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
        self.weight_df = pd.DataFrame({"weight":self.first_layer.weight.cpu().detach().numpy().reshape(-1)})
        # weight update를 위한 데이터프레임 이를 사용해야지 인덱스가 바뀌는 문제를 고정된 인덱스로 해결 가능
    def forward(self,x):
        return self.first_layer(x) 
    def reset_forward(self,x,train_idx,prev_idx): # idx는 attribution이 높은 것들의 index 리스트
        if train_idx == None: 
            return self.forward(x) # forward를 호출 클래스 내의 호출은 무조건 self를 붙여야 된다.
        weight_tensor = self.first_layer.weight
#         print(len(prev_idx),weight_tensor.size())
        self.weight_df.iloc[prev_idx] = weight_tensor.cpu().detach().numpy().reshape(-1,1)
        weight_arr = self.weight_df.iloc[train_idx].values.reshape(-1) # index에 해당하는 array만 뽑는다.
        weight_tensor = torch.FloatTensor(weight_arr).view(self.pred_len,-1) # 해당하는 인덱스만 뽑은후 텐서화
        
        self.first_layer = nn.Linear(len(train_idx),self.pred_len)
        self.first_layer.weight = nn.Parameter(weight_tensor)
        self.first_layer.weight.requires_grad = True
        self.first_layer.to(device)
        return self.first_layer(x[:,:,train_idx]) # 새로 설정한 layer에 input을 집어넣음

weight를 뽑으면 순서대로 정렬 돼있을거임 

In [19]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 100
criterion = SMAPE()

In [20]:
n = len(training)

In [21]:
X_test.size()

torch.Size([1, 1, 1440])

In [22]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
nt_ig_attr_test = np.zeros(sequence_length) # 이렇게 설정해놓음으로써 인덱스가 바뀌지 않도록 조정
temp_df = pd.DataFrame({"attribution":nt_ig_attr_test}) 
train_index = None # 초기 train_index
prev_index = None
for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    # check attribution on mid
    if epoch != 0:
        if epoch == 1: 
            train_index = list(np.arange(sequence_length))
        
        ig = IntegratedGradients(model)        
        nt_ig = NoiseTunnel(ig)
        
        nt_ig_attr_test = nt_ig.attribute(X_test[:,:,train_index])
        
        # X_test는 -split-sequence_length:-split 까지인 validation set과 같다.
        nt_ig_attr_test = nt_ig_attr_test.cpu().detach().numpy().reshape(-1)
#        print(nt_ig_attr_test.shape)
        temp_df['attribution'].iloc[train_index] = nt_ig_attr_test
#         print(temp_df)
        back_df = temp_df['attribution'].iloc[train_index] #인덱스가 매 epoch마다 바뀐다.
        # 처음에 train_index = np.arange(sequence_length) 여야한다.
        
##################################################################        
#        lower_percentile = back_df.quantile(0.01) # 인자[하이퍼파라미터]
##################################################################        
#         standard_mean = (back_df.mean()) 
#         standard_std = back_df.std()
#         lower_bound = standard_mean-standard_std*0.05
#         upper_bound = standard_mean+standard_std*0.05

## 이런식으로 수행하면 포함되지 않을 수 있다.
##################################################################
#         standard_median = (back_df.median()) 
#         standard_std = back_df.std()
#         lower_bound = standard_median-standard_std*0.05
#         upper_bound = standard_median+standard_std*0.05

## median값이 유의미한 경우 제외되는 문제가 발생한다.
##################################################################
        standard_std = back_df.std()
        lower_bound = 0 - standard_std
        upper_bound = 0 + standard_std
## loss값이 그대로 유지되는 현상이 발생
## 해당하는 인덱스가 한번만 사라진이후 계속 그대로 유지된다.
###################################################################
        prev_index = train_index # 이전 인덱스 저장 (모델 파라미터 df 관리를 위해서)

###################################################################
#        train_index = list(back_df[(back_df > lower_percentile)].index)
###################################################################
        train_index = list(back_df[(back_df > upper_bound)|(back_df < lower_bound)].index)
#       print("train",len(train_index))
        # temp_df에 맞는 train_index 추출

#        print(len(train_index))
    for inputs, labels in training:
        # forward pass
        inputs = inputs
        #print(epoch,train_index)
        outputs = model.reset_forward(inputs,train_idx = train_index,prev_idx = prev_index) # train_index를 batch에 대해서 계속 수행한다.
        # prev_idx를 train_idx로 교체 prev_idx로 DataFrame 변경했으므로
        prev_index = train_index
        # defining loss
        loss = criterion(outputs, labels)
        # zero the parameter gradients
        optimizer.zero_grad()
        # computing gradients
        loss.backward()
        
        # accumulating running loss
        running_loss += loss.item()
        # updated weights based on computed gradients
        optimizer.step()
    if epoch % 20 == 0:    
        print('Epoch [%d]/[%d] running accumulative loss across all batches: %.3f' %(epoch + 1, num_epochs, (running_loss/n)*100))


Epoch [1]/[100] running accumulative loss across all batches: 42.193
Epoch [21]/[100] running accumulative loss across all batches: 34.173
Epoch [41]/[100] running accumulative loss across all batches: 34.173
Epoch [61]/[100] running accumulative loss across all batches: 34.173
Epoch [81]/[100] running accumulative loss across all batches: 34.173


In [23]:
len(train_index)

809

In [24]:
pred = model(X_test[:,:,train_index])
pred= pred.cpu().detach().view(-1,1)
nine_result = criterion(pred,target).item()*100
nine_result

55.232709646224976